In [1]:
from data_import import *
from data_preprocess import *

In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('../NLP Topic Recognition')
from topic_recognition import *
#from NLP_class import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [ ]:
from tensorflow import keras
model = keras.models.load_model('../NLP Model Training')

In [ ]:
model.summary()

In [3]:
txt_folder = Path('../Data/textual_raw').rglob('*.csv')
paths = [x for x in txt_folder]
i = 1
df = pd.DataFrame()
# btc_data = preprocess_BTC()
# _, flagged_usernames = flag_users()
flagged_usernames = []
start = timer()
for path in paths:

    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
#     old
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)

    text = NLP_preprocess(text_data)
    text.preprocess_data()
    text.import_tokenizer()
    df_padded = text.tokenize_and_pad()

    text_data["sentiment"] = (model.predict(df_padded))
    text_data["hour"] =  text_data.date.map(lambda x: x[:-6])
    timetable = pd.DataFrame(text_data.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
    timetable = timetable[:-1]
    
    btc_connected = connect_datasets(text_data, btc_data)[1:]
    
    i += 1
    
    if len(timetable.index) != len(btc_connected.index):
        continue
        
    timetable.index = btc_connected.index

    timetable = pd.concat([timetable['Bitcoin'], btc_connected['Open']], axis = 1)
    
    df = pd.concat([df, timetable], axis = 0)

# df.to_csv("Dataset_sentiment.csv")
print(timer() - start)


Iterration 1 of 65
              Bitcoin Ethereum Both   All
2017-09-13 00    1053        0   60  1122
2017-09-13 01     947        0   57  1019
2017-09-13 02    1102        0   73  1181
2017-09-13 03    1202        0   68  1274
2017-09-13 04    1466        0   99  1572
2017-09-13 05    1091        1   82  1186
2017-09-13 06    1106        0   97  1208
2017-09-13 07    1286        0   82  1380
2017-09-13 08    1304        1   84  1402
2017-09-13 09    1329        1   86  1425
2017-09-13 10    1640        0  107  1767
2017-09-13 11    1952        0  126  2100
2017-09-13 12    2051        0  134  2206
2017-09-13 13    1868        0  189  2071
2017-09-13 14    1989        1  197  2200
2017-09-13 15    1975        1  125  2119
2017-09-13 16    1967        0  145  2129
2017-09-13 17    1697        0  132  1846
2017-09-13 18    1593        0  123  1738
2017-09-13 19    1443        2  141  1610
2017-09-13 20    1368        1  112  1520
2017-09-13 21    1314        0  134  1463
2017-09-13 22  

NameError: name 'btc_data' is not defined